In [7]:
%%sql
-- Créer une vue temporaire de la table Bronze
CREATE OR REPLACE TEMPORARY VIEW bronze_wind_power AS
SELECT *
FROM Eolienne.LH_Wind_Power_Bronze.dbo.wind_power;

StatementMeta(, edcfaf9e-6fab-4740-a099-b7160b9e6088, 8, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [12]:
%%sql
-- Nettoyer et enrichir les données
CREATE OR REPLACE TEMPORARY VIEW transformed_wind_power AS
SELECT
    production_id,
    date,
    turbine_name,
    capacity,
    location_name,
    latitude,
    longitude,
    region,
    status,
    responsible_department,
    wind_direction,
    
    -- 🔢 Arrondi des valeurs numériques
    ROUND(wind_speed, 2) AS wind_speed,
    ROUND(energy_produced, 2) AS energy_produced,
    
    -- 📅 Extraction des composants de date
    DAY(date) AS day,
    MONTH(date) AS month,
    QUARTER(date) AS quarter,
    YEAR(date) AS year,
    
    -- 🕐 Correction du format de time
    REGEXP_REPLACE(time, '-', ':') AS time,
    
    -- ⏰ Extraction des composants de temps
    CAST(SUBSTRING(time, 1, 2) AS INT) AS hour_of_day,
    CAST(SUBSTRING(time, 4, 2) AS INT) AS minute_of_hour,
    CAST(SUBSTRING(time, 7, 2) AS INT) AS second_of_minute,
    
    -- 🌅 Calcul de la période de la journée
    CASE
        WHEN CAST(SUBSTRING(time, 1, 2) AS INT) BETWEEN 5 AND 11 THEN 'Morning'
        WHEN CAST(SUBSTRING(time, 1, 2) AS INT) BETWEEN 12 AND 16 THEN 'Afternoon'
        WHEN CAST(SUBSTRING(time, 1, 2) AS INT) BETWEEN 17 AND 20 THEN 'Evening'
        ELSE 'Night'
    END AS time_period
    
FROM bronze_wind_power;

StatementMeta(, edcfaf9e-6fab-4740-a099-b7160b9e6088, 13, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [15]:
%%sql

SELECT * from transformed_wind_power;

StatementMeta(, edcfaf9e-6fab-4740-a099-b7160b9e6088, 16, Finished, Available, Finished)

<Spark SQL result set with 432 rows and 22 fields>

In [16]:
%%sql
-- Supprimer l'ancienne table Silver si elle existe
DROP TABLE IF EXISTS Eolienne.LH_Wind_Power_Silver.dbo.wind_power;

StatementMeta(, edcfaf9e-6fab-4740-a099-b7160b9e6088, 17, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [17]:
%%sql
-- Créer la nouvelle table Silver avec les données transformées
CREATE TABLE Eolienne.LH_Wind_Power_Silver.dbo.wind_power
USING delta
AS
SELECT * FROM transformed_wind_power;

StatementMeta(, edcfaf9e-6fab-4740-a099-b7160b9e6088, 18, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [18]:
%%sql
-- Vérifier que la table a été créée avec succès
SELECT 
    COUNT(*) as total_rows,
    MIN(date) as min_date,
    MAX(date) as max_date,
    COUNT(DISTINCT turbine_name) as turbine_count
FROM Eolienne.LH_Wind_Power_Silver.dbo.wind_power;

StatementMeta(, edcfaf9e-6fab-4740-a099-b7160b9e6088, 19, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [20]:
%%sql
-- Vérifier que la table a été créée avec succès
SELECT *
FROM Eolienne.LH_Wind_Power_Silver.dbo.wind_power;

StatementMeta(, edcfaf9e-6fab-4740-a099-b7160b9e6088, 21, Finished, Available, Finished)

<Spark SQL result set with 432 rows and 22 fields>